# Import software libraries

In [1]:
# Import required libraries.
import sys           # Read system parameters.
import pandas as pd  # Manipulate and analyze data.
import sqlite3       # Manage SQL databases.

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- pandas {}'.format(pd.__version__))
print('- sqlite3 {}'.format(sqlite3.sqlite_version))

Libraries used in this project:
- Python 3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 23:03:20) 
[GCC 7.3.0]
- pandas 1.1.3
- sqlite3 3.30.1


# Examine the database

In [2]:
# Connect to SQLite database.

conn = sqlite3.connect('/home/jovyan/work/Project/data/prod_sample.db')
conn

In [3]:
# List all the tables in the database.

tables = pd.read_sql("""SELECT name 
                          FROM sqlite_master 
                              WHERE type ='table';""", conn)


tables

,name
0,stock_description
1,online_retail_history


# Read data from the `online_retail_history` table

In [4]:
# Write the query to be executed that selects everything from the online_retail_history table.
query = 'SELECT * FROM online_retail_history'



# Use the read_sql function in pandas to read a query into a DataFrame.

online = pd.read_sql(query,conn)

# Preview the first five rows of the data.

online.head()

,Invoice,StockCode,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,536365,85123A,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,536367,84879,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,536373,85123A,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,536375,85123A,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,536378,20725,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


In [5]:
# Get the shape of the data.

online.shape

(15321, 8)

# Read data from the `stock_description` table

In [6]:
# Write the query to be executed that selects everything from the online_retail_history table.
query = 'SELECT * FROM stock_description'



# Use the read_sql function in pandas to read a query into a DataFrame.
stock = pd.read_sql(query,conn)


# Preview the first five rows of the data.

stock.head()

,StockCode,Description
0,10002,INFLATABLE POLITICAL GLOBE
1,10080,GROOVY CACTUS INFLATABLE
2,10120,DOGGY RUBBER
3,10123C,HEARTS WRAPPING TAPE
4,10124A,SPOTS ON RED BOOKCOVER TAPE


In [7]:
# Get the shape of the data.
stock.shape


(3952, 2)

# Aggregate the `online_retail_history` and `stock_description` datasets

In [8]:
# Write a query to aggregate the two datasets so that you have the stock descriptions as well as the stock code.

query = """SELECT Invoice,
                  o.StockCode,
                  s.Description,
                  Quantity,
                  InvoiceDate,
                  Price,
                  CustomerID,
                  Country,
                  TotalAmount
            FROM online_retail_history o
            LEFT JOIN stock_description s
            ON o.StockCode = s.StockCode;"""




# Use the read_sql function in pandas to read a query into a DataFrame.
stock_df = pd.read_sql(query,conn)


# Preview the first five rows of the data.

stock_df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,536365,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,536373,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,536375,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,536378,20725,LUNCH BAG RED RETROSPOT,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


In [9]:
# Get the shape of the data.

stock_df.shape

(17032, 9)

# Identify and fix corrupt or unusable data

In [15]:
# Check the value counts of the "Description" field.

stock_df['Description'].unique()

array(['CREAM HANGING HEART T-LIGHT HOLDER',
       'ASSORTED COLOUR BIRD ORNAMENT', 'LUNCH BAG RED RETROSPOT',
       'PACK OF 72 RETROSPOT CAKE CASES', 'JUMBO BAG RED RETROSPOT',
       'POPCORN HOLDER', 'LUNCH BAG  BLACK SKULL.', '?',
       'REGENCY CAKESTAND 3 TIER', 'PARTY BUNTING',
       'SET OF 3 CAKE TINS PANTRY DESIGN '], dtype=object)

In [10]:
# Remove rows where "Description" is just a question mark (?).

stock_df.drop(stock_df.index[stock_df["Description"] == '?'], inplace = True)





# Preview the first five rows of the data.

stock_df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,536365,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,536373,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,536375,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,536378,20725,LUNCH BAG RED RETROSPOT,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


# Identify and remove duplicates

In [11]:
# Identify all duplicated data.
duplicated = \
stock_df[stock_df.duplicated(keep = False)]


In [12]:
# Print the duplicated data.

print('Number of duplicated rows:',
     duplicated.shape[0])

Number of duplicated rows: 223


In [13]:
# Remove the duplicated data.

stock_df.drop_duplicates(keep = False, inplace = True)





# Preview the first five rows of the data.

stock_df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,536365,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,536373,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,536375,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,536378,20725,LUNCH BAG RED RETROSPOT,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


# Correct date formats

In [14]:
# Get the data types for every column in the DataFrame.

stock_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15098 entries, 0 to 17031
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Invoice      15098 non-null  object 
 1   StockCode    15098 non-null  object 
 2   Description  15098 non-null  object 
 3   Quantity     15098 non-null  int64  
 4   InvoiceDate  15098 non-null  object 
 5   Price        15086 non-null  float64
 6   CustomerID   12327 non-null  object 
 7   Country      15098 non-null  object 
 8   TotalAmount  15086 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 1.2+ MB


In [15]:
# Convert "InvoiceDate" to a "%Y-%m-%d" datetime format.

stock_df['InvoiceDate'] =\
pd.to_datetime(stock_df['InvoiceDate'],
                  format = '%Y-%m-%d')


In [16]:
# Get the data types for every column in the converted DataFrame.

stock_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15098 entries, 0 to 17031
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Invoice      15098 non-null  object        
 1   StockCode    15098 non-null  object        
 2   Description  15098 non-null  object        
 3   Quantity     15098 non-null  int64         
 4   InvoiceDate  15098 non-null  datetime64[ns]
 5   Price        15086 non-null  float64       
 6   CustomerID   12327 non-null  object        
 7   Country      15098 non-null  object        
 8   TotalAmount  15086 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 1.2+ MB


# Examine the table before finishing

In [17]:
# Preview the first five rows of the data.

stock_df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,536365,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,536373,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,536375,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,536378,20725,LUNCH BAG RED RETROSPOT,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


# Load the dataset into a pickle file

In [18]:
# Save the dataset as a pickle file named online_history_cleaned.pickle.

stock_df.to_pickle('stock_df.pickle')



In [19]:
#confirm the date saved as a pickle file


pd.read_pickle('stock_df.pickle').head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country,TotalAmount
0,536365,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,u1785,United Kingdom,15.30
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,u13047,United Kingdom,54.08
2,536373,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:02:00,2.55,u1785,United Kingdom,15.30
3,536375,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2010-12-01 09:32:00,2.55,u1785,United Kingdom,15.30
4,536378,20725,LUNCH BAG RED RETROSPOT,10,2010-12-01 09:37:00,1.65,u14688,United Kingdom,16.50


In [20]:
# Close any connections to the database.

conn.close()